In [1]:
from pyomo.environ import (
    ConcreteModel,
    TransformationFactory,
    assert_optimal_termination,
    units as pyunits,
)
from pyomo.network import Arc

from idaes.core import FlowsheetBlock
import idaes.core.util.scaling as iscale
from idaes.core.util.testing import initialization_tester
from idaes.core.util.initialization import propagate_state
from idaes.models.unit_models import Feed, Product
from idaes.core.util.model_statistics import degrees_of_freedom

from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.core.control_volume_isothermal import ControlVolume0DBlock
from watertap.core.solvers import get_solver
from models.pump_detailed import Pump, VariableEfficiency,PumpCurveDataType
from watertap.core.util.model_diagnostics.infeasible import *

In [10]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()
m.fs.unit = Pump(
    property_package=m.fs.properties,
    variable_efficiency=VariableEfficiency.Flow,
    pump_curve_data_type=PumpCurveDataType.SurrogateCoefficent
)

In [3]:
# Test conditions - 1
feed_flow_vol = 0.126 * pyunits.m**3 / pyunits.s
pump_head = 59.5576 * pyunits.m
density = 1000 * pyunits.kg / pyunits.m**3

# Calculated feed conditions
feed_flow_mass = feed_flow_vol * density
feed_mass_frac_TDS = 0.035

feed_pressure_in = 101325 * pyunits.Pa
feed_pressure_out = feed_pressure_in + pump_head * density * 9.81 * pyunits.m / pyunits.s**2
feed_temperature = 273.15 + 25

feed_mass_frac_H2O = 1 - feed_mass_frac_TDS
m.fs.unit.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(
    feed_flow_mass * feed_mass_frac_TDS
)
m.fs.unit.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(
    feed_flow_mass * feed_mass_frac_H2O
)
m.fs.unit.inlet.pressure[0].fix(feed_pressure_in)
m.fs.unit.inlet.temperature[0].fix(feed_temperature)

m.fs.unit.design_head.fix(pump_head)

m.fs.unit.system_curve_geometric_head.fix(4.57)
m.fs.unit.ref_speed_fraction.fix(1.0)
print(degrees_of_freedom(m))

iscale.calculate_scaling_factors(m)
m.fs.unit.initialize_build()

0
2026-02-12 08:29:53 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.unit.control_volume.properties_in[0.0].flow_mass_phase_comp[Liq,H2O]
2026-02-12 08:29:53 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.unit.control_volume.properties_in[0.0].flow_mass_phase_comp[Liq,TDS]
2026-02-12 08:29:53 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.unit.control_volume.properties_out[0.0].flow_mass_phase_comp[Liq,H2O]
2026-02-12 08:29:53 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.unit.control_volume.properties_out[0.0].flow_mass_phase_comp[Liq,TDS]
2026-02-12 08:29:53 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.unit.control_volume.work
2026-02-12 08:29:53 [INFO] idaes.init.fs.unit.control_volume.properties_out: fs.unit.control_volume.properties_out State Released.
2026-02-12 08:29:53 [INFO] idaes.init.fs.unit.control_volume: Initialization Complete
2026-02-12 08:29:53 [INFO] idaes.init.fs.unit.control

In [7]:
# Test conditions - 2
feed_flow_vol = 0.126 * pyunits.m**3 / pyunits.s
design_speed_fraction = 0.8183
density = 1000 * pyunits.kg / pyunits.m**3

# Calculated feed conditions
feed_flow_mass = feed_flow_vol * density
feed_mass_frac_TDS = 0.035

feed_pressure_in = 101325 * pyunits.Pa
feed_temperature = 273.15 + 25

feed_mass_frac_H2O = 1 - feed_mass_frac_TDS
m.fs.unit.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(
    feed_flow_mass * feed_mass_frac_TDS
)
m.fs.unit.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(
    feed_flow_mass * feed_mass_frac_H2O
)
m.fs.unit.inlet.pressure[0].fix(feed_pressure_in)
m.fs.unit.inlet.temperature[0].fix(feed_temperature)

m.fs.unit.design_speed_fraction.fix(design_speed_fraction)
m.fs.unit.system_curve_geometric_head.fix(4.57)
m.fs.unit.ref_speed_fraction.fix(1.0)
degrees_of_freedom(m)

iscale.calculate_scaling_factors(m)
m.fs.unit.initialize_build()

2026-02-12 08:30:10 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.unit.control_volume.properties_in[0.0].flow_mass_phase_comp[Liq,H2O]
2026-02-12 08:30:10 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.unit.control_volume.properties_in[0.0].flow_mass_phase_comp[Liq,TDS]
2026-02-12 08:30:10 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.unit.control_volume.properties_out[0.0].flow_mass_phase_comp[Liq,H2O]
2026-02-12 08:30:10 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.unit.control_volume.properties_out[0.0].flow_mass_phase_comp[Liq,TDS]
2026-02-12 08:30:10 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.unit.control_volume.work
2026-02-12 08:30:10 [INFO] idaes.init.fs.unit.control_volume.properties_out: fs.unit.control_volume.properties_out State Released.
2026-02-12 08:30:10 [INFO] idaes.init.fs.unit.control_volume: Initialization Complete
2026-02-12 08:30:10 [INFO] idaes.init.fs.unit.control_v

In [11]:
# Test conditions - 3
pump_head = 59.5576 * pyunits.m
design_speed_fraction = 0.8184
density = 1000 * pyunits.kg / pyunits.m**3

feed_flow_vol = 0.126 * pyunits.m**3 / pyunits.s
ref_head = pump_head/design_speed_fraction**2
feed_flow_mass = feed_flow_vol * density
feed_mass_frac_TDS = 0.035

feed_pressure_in = 101325 * pyunits.Pa
feed_pressure_out = feed_pressure_in + pump_head * 1000 * 9.81 * pyunits.kg / pyunits.m**2 / pyunits.s**2
feed_temperature = 273.15 + 25

feed_mass_frac_H2O = 1 - feed_mass_frac_TDS
m.fs.unit.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(
    feed_flow_mass * feed_mass_frac_TDS
)
m.fs.unit.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(
    feed_flow_mass * feed_mass_frac_H2O
)

m.fs.unit.inlet.pressure[0].fix(feed_pressure_in)
m.fs.unit.inlet.temperature[0].fix(feed_temperature)
m.fs.unit.design_head.fix(pump_head)

m.fs.unit.system_curve_geometric_head.fix(4.57)
m.fs.unit.ref_speed_fraction.fix(1.0)

print(degrees_of_freedom(m))

m.fs.unit.initialize_build()


0
2026-02-12 08:30:19 [INFO] idaes.init.fs.unit.control_volume.properties_out: fs.unit.control_volume.properties_out State Released.
2026-02-12 08:30:19 [INFO] idaes.init.fs.unit.control_volume: Initialization Complete
2026-02-12 08:30:19 [INFO] idaes.init.fs.unit.control_volume.properties_in: fs.unit.control_volume.properties_in State Released.
2026-02-12 08:30:19 [INFO] idaes.init.fs.unit: Initialization Complete: optimal - Optimal Solution Found


In [12]:
m.fs.unit.design_speed_fraction.fix(design_speed_fraction)
m.fs.unit.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].unfix()
m.fs.unit.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].unfix()
m.fs.unit.control_volume.properties_in[0].mass_frac_phase_comp['Liq','TDS'].fix()

print(degrees_of_freedom(m))
solver = get_solver()
results = solver.solve(m)
assert_optimal_termination(results)

0


In [ ]:
m.fs.unit.control_volume.display()

In [8]:
solver = get_solver()
results = solver.solve(m)
assert_optimal_termination(results)

In [13]:
# Checking results
print(f"K value: {m.fs.unit.system_curve_flow_constant.value:.4f}")
print(f"Reference flow: {m.fs.unit.ref_flow.value:.4f}")
print(f"Reference head: {m.fs.unit.ref_head.value:.4f}")
print(f"Reference efficiency: {m.fs.unit.ref_efficiency.value:.4f}")
print(f"Design flow: {m.fs.unit.design_flow.value:.4f}")
print(f"Design head: {m.fs.unit.design_head.value:.4f}")
print(f"Design speed fraction: {m.fs.unit.design_speed_fraction.value:.4f}")
print(f"Design efficiency: {m.fs.unit.design_efficiency.value:.4f}")
print(f"Overall efficiency: {m.fs.unit.efficiency_pump[0].value:.4f}")

K value: 3624.9672
Reference flow: 0.1505
Reference head: 86.6682
Reference efficiency: 0.7724
Design flow: 0.1232
Design head: 59.5576
Design speed fraction: 0.8184
Design efficiency: 0.7570
Overall efficiency: 0.6832


In [ ]:
# Count constraints and variables
from pyomo.environ import Var, Constraint

total_constraints = 0
for con in m.fs.unit.component_data_objects(Constraint, active=True, descend_into=True):
    total_constraints += 1

total_vars = 0
unfixed_vars = 0
for var in m.fs.unit.component_data_objects(Var, active=True, descend_into=True):
    total_vars += 1
    if not var.fixed:
        unfixed_vars += 1

print(f"Total variables: {total_vars}")
print(f"Fixed variables: {total_vars - unfixed_vars}")
print(f"Unfixed variables: {unfixed_vars}")
print(f"Total constraints: {total_constraints}")
print(f"DOF: {unfixed_vars - total_constraints}")

In [ ]:
# Check specific pump design variables
pump_vars = ['design_flow', 'design_head', 'design_efficiency', 'design_speed_fraction',
             'system_curve_geometric_head', 'system_curve_flow_constant',
             'ref_flow', 'ref_head', 'ref_efficiency', 'ref_speed_fraction']

print("Pump variable status:")
for var_name in pump_vars:
    if hasattr(m.fs.unit, var_name):
        var = getattr(m.fs.unit, var_name)
        print(f"  {var_name}: fixed={var.fixed}, value={var.value}")